In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import numpy as np
import gala as ga

import astropy.coordinates as coord
import astropy.units as u
from astropy.visualization import quantity_support
quantity_support()
from astropy.table import Table, join

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 24

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.9 * fs,
          'ytick.labelsize': 0.9 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)

In [ ]:
apogee = Table.read("../src/data/allStarLite-dr17-synspec_rev1.fits", hdu=1)

In [ ]:
gaia = Table.read("../src/data/allStar-dr17-synspec-gaiaedr3-xm.fits", hdu=1)

In [ ]:
tbl = join(apogee, gaia, keys="APOGEE_ID")

In [ ]:
clean_tbl = tbl[
      (tbl['LOGG'] > 1) & (tbl['LOGG'] < 5.5)  # realistic spectroscopic stellar parameters (surface gravity)
    & (tbl['TEFF'] > 3500) & (tbl['TEFF'] < 8000)  # same (effective / surface temperature)
    & (tbl['M_H'] > -2) & (tbl['M_H'] < 0.6)  # realistic "bulk metallicity" measurement
    & (tbl['ALPHA_M'] > -1) & (tbl['ALPHA_M'] < 1)  # realistic alpha abundance measurement
    & ((tbl['parallax'] / tbl['parallax_error']) > 8)  # well-measured parallax - the 8 is arbitrary!
]

In [ ]:
fig, ax = plt.subplots()

counts, xedges, yedges, im = ax.hist2d(clean_tbl["M_H"], clean_tbl["ALPHA_M"], bins=50, norm=LogNorm(), cmap="Blues")
cbar = fig.colorbar(im, ax=ax, label="Number of objects")

ax.set_xlabel("[Fe/H] [dex]")
ax.set_ylabel(r"[$\alpha$/Fe] [dex]")

plt.show()

In [ ]:
halo = clean_tbl["M_H"] < -1.25
high_alpha_disc = np.logical_and(clean_tbl["M_H"] > -1, clean_tbl["ALPHA_M"] > 0.2)
low_alpha_disc = np.logical_and(clean_tbl["M_H"] > -1, clean_tbl["ALPHA_M"] < 0.1)

In [ ]:
galcen_frame = coord.Galactocentric()

In [ ]:
fig, ax = plt.subplots()
pot = ga.potential.MilkyWayPotential()


for mask, label in zip([halo, high_alpha_disc, low_alpha_disc], ["Halo", r"High-$\alpha$ Disc", r"Low-$\alpha$ Disc"]):
    smaller_tbl = clean_tbl[mask]
    some_object = smaller_tbl[0]
    print(x)
    c = coord.SkyCoord(
        ra=some_object["ra"],
        dec=some_object["dec"],
        unit=(u.deg, u.deg),
        distance=(some_object["parallax"] * u.mas).to(u.arcsecond).value**(-1) * u.pc,
        pm_ra_cosdec=some_object["pmra"] * u.mas/u.yr * np.cos(some_object["dec"] * u.deg),
        pm_dec=some_object["pmdec"]*u.mas/u.yr,
        radial_velocity=some_object["dr2_radial_velocity"]*u.km/u.s
    )
    
    pos = ga.dynamics.PhaseSpacePosition(c.transform_to(galcen_frame).data)
    orbits = pot.integrate_orbit(pos, t1=0*u.Gyr, t2=10*u.Gyr, n_steps=4000)
    _ = orbits.cylindrical.plot(['rho', 'z'], axes=[ax], label=label)

leg = ax.legend(ncol=3, title="Random objects from APOGEE/Gaia", loc="upper center")
leg.get_title().set_fontsize(0.7*fs)
    
ax.set_xlim(0, 13)
ax.set_ylim(-4.5, 5.5)
    
plt.show()